In [1]:
import numpy as np
import cv2
import glob
import os

class MyImage:
    def __init__(self, src_img):
        self.src_img = src_img
        self.gray_img = np.zeros((self.src_img.shape[0],self.src_img.shape[1]), dtype=int)
        self.mtb_img  = np.zeros((self.src_img.shape[0],self.src_img.shape[1]), dtype=int)
        self.mask_img = np.zeros((self.src_img.shape[0],self.src_img.shape[1]), dtype=int)
        self.denoise_img = np.zeros((self.src_img.shape[0],self.src_img.shape[1]), dtype=int)
        self.med_i = 0
        self.threshold_i = 15
    
    def grayscale(self):
        self.src_img = self.src_img.astype(np.int)
        self.gray_img = (self.src_img[:,:,0]*19/256 + self.src_img[:,:,1]*183/256 + self.src_img[:,:,2]*54/256)
        self.med_i = np.median(self.gray_img)

    def mtb(self):
        # # convert to MTB
        self.mtb_img = np.where(self.gray_img > self.med_i, 255, 0)
    
    def mask(self):
        # # exclusion map
        upper_bound = self.med_i + self.threshold_i
        lower_bound = self.med_i - self.threshold_i
        self.mask_img[self.gray_img < lower_bound] = 255
        self.mask_img[self.gray_img > upper_bound] = 255
     
    def bind_mtb_mask(self):
        # bind mask_img and mtb_img
        self.denoise_img = np.bitwise_and(self.mtb_img, self.mask_img)

    def convert_image(self):
        self.grayscale()
        self.mtb()
        self.mask()
        self.bind_mtb_mask()

# Read file


imgspath = glob.glob(os.path.join('data','*.png'))
imgs = [cv2.imread(i) for i in imgspath]

for i,ele in enumerate(imgs):
    myimage = MyImage(ele)
    myimage.convert_image()
    cv2.imwrite(os.path.join('output', f'img{i}.jpg'), myimage.denoise_img)
    

img = cv2.imread('data/memorial0061.png')
myimage = MyImage(img)
myimage.grayscale()
print('g')
myimage.mtb()
print('m')
myimage.mask()
print('mask')
myimage.bind_mtb_mask()
print('b')

# myimage.convert_image()
cv2.imwrite('img.jpg', myimage.denoise_img)


C:\Users\HanWen\AppData\Local\Temp\ipykernel_7000\3705129500.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.src_img = self.src_img.astype(np.int)


g
m
mask
b


True

In [35]:
cv_img = []
i = 0
for img in glob.glob("data/*.png"):
    n = cv2.imread(img)
    myimage = MyImage(n)
    # myimage.convert_image()
    cv2.imwrite('de' +i.__str__+ '.png', n)
    print('finish' + img)
    i+=1
    #cv_img.append(myimage)
    
#print(cv_img) 

TypeError: can only concatenate str (not "method-wrapper") to str